# Imports 

## Application-specific imports 

In [ ]:
import sys

In [ ]:
sys.path.append("../config/")
import config

In [ ]:
sys.path.append("../metaflow/")
import preprocess_fn

## General 

In [ ]:
import pickle
import itertools

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing

In [ ]:
import pyspark
import pyspark.sql.functions as fn
import pyspark.sql.types as t
from pyspark.ml import Pipeline
from pyspark.ml.feature import (Normalizer, Tokenizer, HashingTF, IDF)
from pyspark.mllib.linalg.distributed import (IndexedRow, IndexedRowMatrix)

# Load data from parquet

In [ ]:
df = spark.read.parquet(f'{config.ARTIFACTS}/dataset/M20_cards.parquet')

# Preprocess 

In [ ]:
df_filtered = df

## Fill in NA 

In [ ]:
df_filtered = df_filtered.na.fill(value="", subset=["filteredText"])

## NLP with Spark

In [ ]:
tokenizer = Tokenizer(inputCol="filteredText", outputCol="words")
wordsData = tokenizer.transform(df_filtered)

In [ ]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
featurizedData = hashingTF.transform(wordsData)

In [ ]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

In [ ]:
rescaledData = idfModel.transform(featurizedData)

In [ ]:
rescaledData.select("number", "name", "features").show()

In [ ]:
pd_number_names = rescaledData.select("number", "name").toPandas()

In [ ]:
rescaledData = rescaledData.select("number", "features")

In [ ]:
pd_rescaledData = rescaledData.select("features").toPandas()

In [ ]:
len(pd_rescaledData)

In [ ]:
dense_features = np.zeros((329, 1000))

In [ ]:
for i in range(329):
    dense_vect = pd_rescaledData['features'][i].toArray()
    dense_features[i, :] = dense_vect

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cos_similarity = cosine_similarity(dense_features)

In [ ]:
cos_similarity = np.corrcoef(dense_features)

In [ ]:
# normalizer = Normalizer(inputCol="features", outputCol="normed_features")
# data = normalizer.transform(rescaledData)

In [ ]:
# mat = IndexedRowMatrix(data.select("number", "normed_features").rdd.map(lambda row: IndexedRow(row.number, row.normed_features.toArray()))).toBlockMatrix()

In [ ]:
# dot = mat.multiply(mat.transpose())

In [ ]:
# dot.numCols(), dot.numRows()

In [ ]:
# cos_similarity = dot.toLocalMatrix().toArray()

In [ ]:
cos_similarity

In [ ]:
len(pd_number_names)

In [ ]:
names_lens = pd_number_names['name'].apply(func=len)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f, axes = plt.subplots(figsize=(25, 25), dpi=300)
c = axes.matshow(cos_similarity)
plt.xticks(ticks=range(329), labels=pd_number_names['name'].tolist(), rotation=90, fontsize=4)
plt.colorbar(c)

In [ ]:
import bct

In [ ]:
cos_similarity = np.abs(cos_similarity)

In [ ]:
ca, Q = bct.community_louvain(1.0 / cos_similarity)

In [ ]:
Q

In [ ]:
m = np.max(ca)

In [ ]:
m

In [ ]:
len(ca)

In [ ]:
for i in range(1, m + 1):
    indices = np.where(ca == i)[0]
    group = pd_number_names.loc[indices]
    print(group.sort_values('name'))
    print("*" * 10)